# Project Model Development

## Sidney Cannon-Bailey
## 11/12/2020

In [12]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
# set current path
# set file location on your local machine
dataLoc = '../data/dataWithFeatures.csv'

In [3]:
# load dataset
df = pd.read_csv(dataLoc)

In [4]:
df = df.dropna().reset_index(drop=True)

# convert confirmed cases strings to numbers
df['Confirmed Cases']= df['Confirmed Cases'].str.replace(',','')
df['Confirmed Cases'] = df['Confirmed Cases'].astype(int)
    

In [7]:
# isolate x and y data
x = df.iloc[:,4:-3]
y = df[['majorityWhite','majorityBlack','raciallyDiverse']]

# scale x data
scaledX = pd.DataFrame(MinMaxScaler().fit_transform(x), columns=list(x))

# add zip codes back to x data
scaledX['ZIP'] = df['ZIP'].copy()

# split into testing and training data
X_train, X_test, y_train, y_test = train_test_split(scaledX, y['raciallyDiverse'], test_size=.2)

In [8]:
# first model: Log regression
# can we predict if a neighborhood is racially diverse using the health and covid data?
logModel = LogisticRegression().fit(X_train,y_train)
logModel.score(X_test, y_test)


C:\Users\sidne\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8620689655172413

In [10]:
svmModel = svm.SVC().fit(X_train, y_train)
svmModel.score(X_test, y_test)

C:\Users\sidne\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.8275862068965517

In [22]:
# find optimum k value
kParameter = {'n_neighbors':[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]};
neigh = KNeighborsClassifier()
neighOpt = GridSearchCV(neigh, kParameter);
neighOpt.fit(X_train, y_train)
neighOpt.best_estimator_

C:\Users\sidne\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=12, p=2,
                     weights='uniform')

In [23]:
# found k=12 was the optimal parameter
knnModel = KNeighborsClassifier(n_neighbors=12)
knnModel.fit(X_train, y_train)
knnModel.score(X_test, y_test)


0.8275862068965517